In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from  collections import Counter
from datetime import datetime

In [ ]:
netflix_df = pd.read_csv('netflix_titles.csv', encoding='latin-1')

In [ ]:
netflix_df.head()

In [ ]:
summary = netflix_df.describe()
print(summary)

In [ ]:
netflix_df.info()

In [ ]:
netflix_df.isnull().sum()

In [ ]:
empty_director = netflix_df[netflix_df['director'].isnull()]
print(empty_director)

In [27]:
cols = ['director', 'cast', 'country']

netflix_df[cols] = netflix_df[cols].fillna('Unknown')
print(netflix_df[cols])
# fill in  mode
netflix_df['date_added'].fillna(netflix_df['date_added'].mode()[0],inplace=True)
netflix_df['rating'].fillna(netflix_df['rating'].mode()[0],inplace=True)
for i in netflix_df.index:
    if netflix_df.loc[i, 'rating']== 'UR':
        netflix_df.loc[i,'rating']= 'NR'
#drop row with missing values
#netflix_df=netflix_df.dropna(subset=['duration'])



             director                                               cast  \
0     Kirsten Johnson                                            Unknown   
1             Unknown  Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...   
2     Julien Leclercq  Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...   
3             Unknown                                            Unknown   
4             Unknown  Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...   
...               ...                                                ...   
8802    David Fincher  Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...   
8803          Unknown                                            Unknown   
8804  Ruben Fleischer  Jesse Eisenberg, Woody Harrelson, Emma Stone, ...   
8805     Peter Hewitt  Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...   
8806      Mozez Singh  Vicky Kaushal, Sarah-Jane Dias, Raaghav Chanan...   

            country  
0     United States  
1      South Africa  
2           Unknown  

In [ ]:
netflix_df.head()

In [ ]:
# Check for duplicate rows
duplicate_rows = netflix_df[netflix_df.duplicated()]

print(duplicate_rows)

In [ ]:
netflix_df = netflix_df.iloc[0:8808:]

In [ ]:
netflix_df.drop_duplicates()

In [ ]:
# Create separate DataFrames for TV Shows and Movies
tv_shows_df = netflix_df[netflix_df['type'] == 'TV Show']
print(tv_shows_df)

In [ ]:
movies_df = netflix_df[netflix_df['type'] == 'Movie']
print(movies_df)

In [24]:
# transfer to datetime type
netflix_df['duration_mins']=netflix_df[netflix_df['type']=='Movie']['duration'].str.extract('(\d+)').astype(float)
netflix_df['duration_seasons']=netflix_df[netflix_df['type']=='TV Show']['duration'].str.extract('(\d+)').astype(float)
#NaN values replace with 0
netflix_df[['duration_mins', 'duration_seasons']]= netflix_df[['duration_mins', 'duration_seasons']].fillna(0)
#drop the  duration column
netflix_df=netflix_df.drop('duration', axis=1)


KeyError: 'duration'

In [25]:
netflix_df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,listed_in,description,duration_mins,duration_seasons
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,Unknown,United States,"September 25, 2021",2020,PG-13,Documentaries,"As her father nears the end of his life, filmm...",90.0,0.0
1,s2,TV Show,Blood & Water,Unknown,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",0.0,2.0
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",Unknown,"September 24, 2021",2021,TV-MA,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,0.0,1.0
3,s4,TV Show,Jailbirds New Orleans,Unknown,Unknown,Unknown,"September 24, 2021",2021,TV-MA,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",0.0,1.0
4,s5,TV Show,Kota Factory,Unknown,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,0.0,2.0
